<a id="use-onnx-model-converted-from-lightgbm"></a>
# Use ONNX model converted from scikit-learn with `ibm-watsonx-ai`

This notebook facilitates `ONNX`, `scikit-learn`, and `watsonx.ai Runtime` service. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository in order to convert the model to ONNX format. It also introduces commands for persisting model, deploying model, and scoring it.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Learning goals

The learning goals of this notebook are:

-  Train a scikit-learn model
-  Convert the native scikit-learn model to ONNX format
- [Optional] perform conversion for a custom scikit-learn model wrapped in a sklearn pipeline
-  Persist the converted model in the watsonx.ai Runtime repository
-  Deploy the model for online scoring using client library
-  Score sample records using the client library

## Contents

This notebook contains the following parts:

1.	[Setting up the environment](#setup)
2.	[Creating a scikit-learn model](#create)
3.	[Creating and converting the scikit-learn model to ONNX format](#convert)
4.	[Persisting the converted ONNX model](#upload)
5.	[Deploying and scoring the ONNX model](#deploy)
6.  [Cleaning up](#cleanup)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
<a id="1-setting-up-the-environment"></a>
## 1. Setting up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [watsonx.ai Runtime](https://cloud.ibm.com/catalog/services/watsonxai-runtime) instance (information on service plans and further reading can be found [here](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp)).

### 1.1. Installing and importing the `ibm-watsonx-ai` and dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1
!pip install scikit-learn==1.3.0 | tail -n 1
!pip install onnxruntime==1.16.3 | tail -n 1
!pip install onnx==1.16.2 | tail -n 1
!pip install skl2onnx | tail -n 1

In [1]:
import getpass
import json
import numpy as np
import onnx
import onnxruntime as ort
from ibm_watsonx_ai import APIClient, Credentials
from skl2onnx import to_onnx, update_registered_converter
from skl2onnx.algebra.onnx_ops import OnnxDiv, OnnxSub
from skl2onnx.common.data_types import guess_numpy_type
from sklearn import datasets
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

### 1.2. Connecting to watsonx.ai Runtime

Authenticate with the watsonx.ai Runtime service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

Get the value of `api_key` from the output.


Location of your watsonx.ai Runtime instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance INSTANCE_NAME
```

Get the value of `location` from the output.

**Tip**: You can generate your `Cloud API key` by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service-specific url by going to the [**Endpoint URLs** section of the watsonx.ai Runtime docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance details.

You can also get the service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key, and paste it below.

**Action**: Enter your `api_key` and `location` in the following cells.

In [2]:
api_key = getpass.getpass("Please enter your api key (hit enter): ")

Please enter your api key (hit enter):  ········


In [3]:
location = "ENTER YOUR LOCATION HERE"

If you are running this notebook on Cloud, you can access the `location` via:

```
location = os.environ.get("RUNTIME_ENV_REGION")
```

In [4]:
credentials = Credentials(
    api_key=api_key,
    url=f'https://{location}.ml.cloud.ibm.com'
)

In [5]:
client = APIClient(credentials=credentials)

### 1.3. Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use the `ibm_watsonx_ai` SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [6]:
space_id = "ENTER YOUR SPACE ID HERE"

You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai Runtime, you need to set **space** which you will be using.

In [8]:
client.set.default_space(space_id)

'SUCCESS'

<a id="create"></a>
## 2. Creating a scikit-learn model
In this section, you will train and sample a scikit-learn model.

Load the iris data first.

In [9]:
iris = datasets.load_iris()
x, y = iris.data, iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

### 2.1. Creating an estimator

In [10]:
model = RandomForestClassifier()

### 2.2. (OPTIONAL) Creating a pipeline with custom Transformer
To create a scikit-learn pipeline consisting of e.g. a custom standard scaler followed by an estimator, change the `Markdown` cell below to a `Code` cell and remove the triple backticks (` ``` `).

```
class CustomStandardScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mean_ = None
        self.scale_ = None

    def fit(self, X, y=None):
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
        self.scale_[self.scale_ == 0] = 1.0
        return self

    def transform(self, X):
        return (X - self.mean_) / self.scale_

model = Pipeline([("transformer", CustomStandardScaler()), ("estimator", model)])
```

### 2.3. Training the model

In [11]:
model.fit(x_train, y_train)

RandomForestClassifier()

<a id="evaluation"></a>
### 2.4. Evaluating the model

In [12]:
model.predict(x_test[:3])

array([1, 0, 2])

<a id="convert"></a>
## 3. Converting the scikit-learn model to ONNX format
In this section, you will convert the model to the ONNX format.

### 3.1. (OPTIONAL) Register custom converter
To convert a CUSTOM scikit learn model to an ONNX format you need to register a custom onnx converter. In order to do so for previously, optionally, defined custom standard scaler, change the `Markdown` cell below to a `Code` cell and remove the triple backticks (```). The example demonstrates an sklearn pipeline with custom transformer, but the steps would remain the same for a standalone custom model.
For more information refer to the [documentation](https://onnx.ai/sklearn-onnx/auto_tutorial/plot_icustom_converter.html).

```
def custom_standard_scaler_shape_calculator(operator):
    operator.outputs[0].type = operator.inputs[0].type

def custom_standard_scaler_converter(scope, operator, container):
    op = operator.raw_operator
    opv = container.target_opset
    out = operator.outputs

    X = operator.inputs[0]

    dtype = guess_numpy_type(X.type)
    Y = OnnxDiv(
        OnnxSub(X, op.mean_.astype(dtype), op_version=opv),
        op.scale_.astype(dtype),
        op_version=opv,
        output_names=out[:1],
    )
    Y.add_to(scope, container)


update_registered_converter(
    CustomStandardScaler,
    "CustomStandardScaler",
    custom_standard_scaler_shape_calculator,
    custom_standard_scaler_converter,
)
```

### 3.2. Converting the model

In [13]:
onnx_model = to_onnx(model, x_train, target_opset=19)

onnx_model_filename = "sklearn_model.onnx"

with open(onnx_model_filename, "wb") as f:
    f.write(onnx_model.SerializeToString())

<a id="evaluation-onnx"></a>
### 3.3. Evaluating the ONNX Model

After exporting the model, you should verify its integrity and ensure that it functions as expected. We will use `onnxruntime` to load the model and perform inference on the test data. Additionally, we’ll use `onnx`'s `checker` module to validate the exported ONNX model.

In [14]:
onnx.checker.check_model(onnx_model)

In [15]:
session = ort.InferenceSession(onnx_model_filename)

input_data = {session.get_inputs()[0].name: x_test[:3].tolist()}
session.run([], input_data)

[array([1, 0, 2], dtype=int64),
 [{0: 0.0, 1: 0.9899999499320984, 2: 0.009999999776482582},
  {0: 0.9899999499320984, 1: 0.009999999776482582, 2: 0.0},
  {0: 0.0, 1: 0.0, 2: 1.0}]]

Every classifier, by design, produces two outputs: the predicted label and the probability distribution across all possible labels.
As you can see, the predicted values are consistent with those calculated in the [evaluation](#evaluation) section.

<a id="upload"></a>
## 4. Persisting the converted ONNX model

In this section, you will learn how to store your converted ONNX model in watsonx.ai Runtime.

### 4.1. Publishing the model in watsonx.ai Runtime repository

Define model name, type and software spec.

In [16]:
sofware_spec_id = client.software_specifications.get_id_by_name("onnxruntime_opset_19")
onnx_model_zip = "sklearn_onnx.zip"

In [17]:
!zip {onnx_model_zip} {onnx_model_filename}

updating: sklearn_model.onnx (deflated 92%)


In [18]:
metadata = {
    client.repository.ModelMetaNames.NAME: 'scikit-learn to ONNX converted model',
    client.repository.ModelMetaNames.TYPE: 'onnxruntime_1.16',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sofware_spec_id
}

published_model = client.repository.store_model(
    model=onnx_model_zip,
    meta_props=metadata
)

### 4.2. Getting model details

In [ ]:
published_model_id = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_id)
print(json.dumps(model_details, indent=2))

<a id="deploy"></a>
## 5. Deploying and scoring the ONNX model

In this section you'll learn how to create an online scoring service and predict on unseen data.

### 5.1. Creating online deployment for published model

In [20]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of scikit-learn to ONNX converted model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '41f34250-e6f0-4f19-933d-da60ccf3dd6e' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
.
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='7629ed62-cdfa-4607-9b29-e0d93d2a7f4f'
-----------------------------------------------------------------------------------------------




In [21]:
deployment_id = client.deployments.get_id(created_deployment)

Now you can print an online scoring endpoint. 

In [ ]:
client.deployments.get_scoring_href(created_deployment)

### 5.2. Getting deployment details

In [ ]:
client.deployments.get_details(deployment_id)

### 5.3. Scoring

You can use the method shown below to perform a test scoring request against the deployed model.

Prepare scoring payload with records to score.

In [24]:
scoring_payload = {
    "input_data": [
        {
            "values": x_test[:3]
        }
    ]
}

Use ``client.deployments.score()`` method to run scoring.

In [25]:
predictions = client.deployments.score(deployment_id, scoring_payload)

Let's print the result of predictions.

In [26]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "id": "output_label",
      "values": [
        1,
        0,
        2
      ]
    },
    {
      "id": "output_probability",
      "values": [
        {
          "0": 0.0,
          "1": 0.9899999499320984,
          "2": 0.009999999776482582
        },
        {
          "0": 0.9899999499320984,
          "1": 0.009999999776482582,
          "2": 0.0
        },
        {
          "0": 0.0,
          "1": 0.0,
          "2": 1.0
        }
      ]
    }
  ]
}


As you can see, the predicted values are consistent with those calculated in the [evaluation](#evaluation-onnx) section.

<a id="cleanup"></a>
## 6. Cleaning up

If you want to clean up after the notebook execution, i.e. remove any created assets like:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
<a id="7-summary-and-next-steps"></a>
## 7. Summary and next steps

 You successfully completed this notebook! You learned how to use ONNX, scikit-learn machine learning library as well as watsonx.ai Runtime for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx)_ for more samples, tutorials, documentation, how-tos, and blog posts.

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.